## Práctica 2: Transformaciones geométricas
Aris Vazdekis Soria y Alejandra Ruiz de Adana Fleitas

-------------------------------------------------------------------------------------------------
- 1a) Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente. Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y. Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro. Hacer escalados uniformes. Es necesario indicar los factores de escala. (Con todos los optativos) ✔️

In [4]:
import cv2
import numpy as np

# Variables globales
dragging = False #Indica si se esta arrastrando la imagen
start_x, start_y = 0, 0 #Coordenadas de inicio de arastre
tx, ty = 0, 0  # Traslación acumulada
rotation_center = None  # Centro de rotación

# Función de manejo de eventos del ratón
def mouse_event(event, x, y, flags, param):
    global start_x, start_y, tx, ty, dragging, rotation_center

    # Establecer el centro de rotación al hacer clic izquierdo
    if event == cv2.EVENT_LBUTTONDOWN:
        rotation_center = (x, y)  # Actualizar el centro de rotación
        print(f'Nuevo centro de rotación: {rotation_center}')

        # Empezar a arrastrar la imagen
        dragging = True
        start_x, start_y = x, y

    # Actualizar la traslación mientras se arrastra
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        dx = x - start_x #Calcula el desplazamiento horizontal desde la última posición donde se inició el arrastre
        dy = y - start_y
        tx += dx #acumulando el desplazamiento horizontal de la imagen, lo que permite que la imagen se desplace a la derecha o izquierda en la ventana.
        ty += dy
        start_x, start_y = x, y

    # Finalizar el arrastre cuando se suelta el botón
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False

# Cargar la imagen
image = cv2.imread('02/images/cats.png')
if image is None:
    print("Error: No se pudo cargar la imagen.")
    exit()

# Tamaño original de la imagen
h, w = image.shape[:2]

cv2.namedWindow('Transformaciones')

# Crear trackbars para las transformaciones
cv2.createTrackbar('Rotacion angulos', 'Transformaciones', 0, 360, lambda x: None)#Hazme una función que se llama x y que haga NONE
cv2.createTrackbar('Escala (100%)', 'Transformaciones', 100, 300, lambda x: None)

# Establecer la función de callback para el mouse
cv2.setMouseCallback('Transformaciones', mouse_event)

while True:
    # Leer los valores de las barras deslizantes
    rotation_angle = cv2.getTrackbarPos('Rotacion angulos', 'Transformaciones')
    scale = cv2.getTrackbarPos('Escala (100%)', 'Transformaciones') / 100.0

    # ------- TRANSLACION -------
    # Crea una matriz de transformación afín para la traslación, donde tx y ty representan el desplazamiento horizontal y vertical, respectivamente
    M_translation = np.float32([[1, 0, tx],
                                [0, 1, ty]]) # tx y ty representan la traslación de la imagen en el eje x e y
                                             # Los 1 representan escalado y/o rotación.
                                             # Los 0 representan el sesgo.
    translated_image = cv2.warpAffine(image, M_translation, (w, h))

    # ------- ESCALADO -------
    scaled_image = cv2.resize(translated_image, None, fx=scale, fy=scale) #None, le estás diciendo que no especifique el tamaño de la imagen de salida directamente.

    # Recortar la imagen si es más grande que el tamaño de la ventana
    if scale > 1.0:
        #Devuelve una tupla que contiene las dimensiones de la imagen escalada
        h_scaled, w_scaled = scaled_image.shape[:2] #se utiliza para obtener solo la altura y el ancho. No queremos colores
        
        # Calcular el recorte
        y_start = max(0, (h_scaled - h)) #Calculamos un punto para no ir hacia arriba. Siempre queremos que sea 0 o más.
        y_end = min(h_scaled, (h_scaled + h)) #Es el punto donde queremos que termine nuestro recorte
        x_start = max(0 , (w_scaled - w))
        x_end = min(w_scaled, (w_scaled + w))

        scaled_image = scaled_image[y_start:y_end, x_start:x_end]

    # ------- ROTACION -------
    # Asegurarse de que el centro de rotación esté dentro de la imagen
    if rotation_center is None:
        rotation_center = (w // 2, h // 2)  # Por defecto, el centro de la imagen

    M_rotation = cv2.getRotationMatrix2D(rotation_center, rotation_angle, 1) #1 para que no cambie de tamaño
    #Aplica la rotación a la imagen escalada, generando
    rotated_image = cv2.warpAffine(scaled_image, M_rotation, (w, h))

    # Mostrar frame transformado
    cv2.imshow('Transformaciones', rotated_image)

    # Salir si se pulsa la tecla Esc
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

-------------------------------------------------------------------------------------------------
- 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen. ✔️

In [7]:
import cv2
import numpy as np

# Variables globales para almacenar los puntos seleccionados
points = []

# Función para manejar los clics del ratón
def select_points(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 4:  # Solo permitir seleccionar 4 puntos
            points.append([x, y])
            print(f'Punto seleccionado: {x}, {y}')
            
            # Dibujar el punto seleccionado y las líneas
            cv2.circle(image_display, (x, y), 5, (0, 255, 0), -1)
            if len(points) > 1:
                cv2.line(image_display, tuple(points[-2]), tuple(points[-1]), (255, 0, 0), 2) #el penultimo punto points[-2], el ultimo punto points[-1]
            if len(points) == 4:
                cv2.line(image_display, tuple(points[-1]), tuple(points[0]), (255, 0, 0), 2)
            cv2.imshow('Imagen', image_display)

        if len(points) == 4:  # Cuando se seleccionen 4 puntos
            perform_projection()

# Función para realizar la proyección cuando se seleccionen 4 puntos
def perform_projection():
    global points
    # Definir los puntos de destino para la proyección
    h, w = image.shape[:2]  # Altura y ancho de la imagen

    #Esto crea cuatro puntos que marcan las esquinas de un rectángulo grande que queremos llenar con nuestra imagen
    pts1 = np.float32([[0, 0], [0, h], [w, h], [w, 0]])  # Definir el tamaño de salida

    # Convertir puntos seleccionados a tipo float32
    pts2 = np.float32(points) #Necesario para operaciones en opencv

    # Calcular la transformación proyectiva
    Tp = cv2.getPerspectiveTransform(pts1, pts2)

    # Aplicar la transformación proyectiva
    imageP = cv2.warpPerspective(image, Tp, (w, h))

    # Mostrar la imagen proyectada
    cv2.imshow('Proyección', imageP)

    # Reiniciar los puntos seleccionados
    points.clear()

# Cargar la imagen
file = r'02/images/cats.png'
image = cv2.imread(file)
image_display = image.copy()  # Copia para mostrar los puntos seleccionados

# Verificar si la imagen se ha cargado correctamente
if image is None:
    print("Error: No se pudo cargar la imagen.")
else:
    # Mostrar la imagen y configurar el callback del ratón
    cv2.imshow('Imagen', image_display)
    cv2.setMouseCallback('Imagen', select_points)

    # Esperar hasta que se cierre la ventana
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

----
- 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz ✔️ !

In [9]:
import cv2
import numpy as np

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    # Generar nueva matriz de cámara a partir de los parámetros
    distCoeff = np.zeros((4, 1), np.float64) #Arreglo de 0
    distCoeff[0, 0] = k1  # Término de primer orden
    distCoeff[1, 0] = k2  # Término de segundo orden

    # Suponer matriz unitaria para la cámara
    cam = np.eye(3, dtype=np.float32) #Matriz 3x3, describe cómo se captura la imagen en relación con el tamaño y la posición del centro de la imagen.
    cam[0, 2] = w / 2.0  # Definir centro x
    cam[1, 2] = h / 2.0  # Definir centro y
    cam[0, 0] = 10.0     # Definir longitud focal x cuán "cerca" o "lejos"
    cam[1, 1] = 10.0     # Definir longitud focal y cuán "cerca" o "lejos"

    # Aplicar distorsión a la imagen
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img

# Cargar una imagen de ejemplo
file = r'02/images/cats.png'
image = cv2.imread(file)

# Verificar si la imagen se ha cargado correctamente
if image is None:
    print("Error: No se pudo cargar la imagen.")
    exit()

# Crear una ventana
cv2.namedWindow('Distorsión')

# Función para actualizar la imagen según los sliders
def update(val):
    k1 = cv2.getTrackbarPos('k1', 'Distorsión')
    k2 = cv2.getTrackbarPos('k2', 'Distorsión')
    distorsion_tipo = cv2.getTrackbarPos('Tipo', 'Distorsión')

    # Ajustar los coeficientes según el tipo de distorsión
    if distorsion_tipo == 0:  # Barril
        k1 = k1 * 0.0001  # Ajustar el rango
        k2 = k2 * 0.0001  # Ajustar el rango
    else:  # Almohadilla
        k1 = -k1 * 0.0001  # Ajustar el rango
        k2 = -k2 * 0.0001  # Ajustar el rango

    distorted_img = apply_distortion(image, k1, k2)
    cv2.imshow('Distorsión', distorted_img)

# Crear trackbars para los coeficientes k1 y k2
cv2.createTrackbar('k1', 'Distorsión', 0, 100, update)
cv2.createTrackbar('k2', 'Distorsión', 0, 100, update)

# Crear un trackbar para seleccionar el tipo de distorsión
# 0 -> Barril, 1 -> Almohadilla
cv2.createTrackbar('Tipo', 'Distorsión', 0, 1, update)

# Mostrar la imagen original
cv2.imshow('Distorsión', image)

# Esperar hasta que se presione una tecla
cv2.waitKey(0)

# Cerrar todas las ventanas de OpenCV
cv2.destroyAllWindows()


-------------------------------------------------------------------------------------------------
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen. (Con todos los optativos) ✔️

In [10]:
import cv2
import numpy as np

dragging = False
start_x, start_y = 0, 0
tx, ty = 0, 0  # Traslación acumulada
rotation_center = None  # Centro de rotación

# Función de manejo de eventos del ratón
def mouse_event(event, x, y, flags, param):
    global start_x, start_y, tx, ty, dragging, rotation_center

    # Establecer el centro de rotación al hacer clic izquierdo
    if event == cv2.EVENT_LBUTTONDOWN:
        rotation_center = (x, y)  # Actualizar el centro de rotación
        print(f'Nuevo centro de rotación: {rotation_center}')

        # Empezar a arrastrar la imagen
        dragging = True
        start_x, start_y = x, y

    # Actualizar la traslación mientras se arrastra
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        dx = x - start_x
        dy = y - start_y
        tx += dx
        ty += dy
        start_x, start_y = x, y

    # Finalizar el arrastre cuando se suelta el botón
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False

# Cargar el vídeo
cap = cv2.VideoCapture(r'02/videos/stanford.avi')
if not cap.isOpened():
    print("Error: No se pudo abrir el video.")
    exit()


cv2.namedWindow('Transformaciones')

# Crear trackbars para las transformaciones
cv2.createTrackbar('Rotacion angulos', 'Transformaciones', 0, 360, lambda x: None)
cv2.createTrackbar('Escala (100%)', 'Transformaciones', 100, 300, lambda x: None)

# Establecer la función de callback para el mouse
cv2.setMouseCallback('Transformaciones', mouse_event)

while True:  # Mantener el bucle para la reproducción continua
    ret, frame = cap.read()
    #cap.read() intenta leer un fotograma del video.
    #La variable ret será True si se ha leído un fotograma correctamente y False si no.
    #frame contendrá la imagen del fotograma leído.
    if not ret: #Se llega al final del video
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reiniciar el video al final
        continue
    
    # Leer los valores de las barras deslizantes
    rotation_angle = cv2.getTrackbarPos('Rotacion angulos', 'Transformaciones')
    scale = cv2.getTrackbarPos('Escala (100%)', 'Transformaciones') / 100.0

    # Tamaño del frame
    h, w = frame.shape[:2]

    # ------- TRANSLACION -------
    M_translation = np.float32([[1, 0, tx], 
                                [0, 1, ty]])
    translated_frame = cv2.warpAffine(frame, M_translation, (w, h))

    # ------- ESCALADO -------
    scaled_frame = cv2.resize(translated_frame, None, fx=scale, fy=scale)

    # Recortar la imagen si es más grande que el tamaño de la ventana
    if scale > 1.0:
        h_scaled, w_scaled = scaled_frame.shape[:2]
        # Calcular el recorte
        y_start = max(0, (h_scaled - h) // 2)
        y_end = min(h_scaled, (h_scaled + h) // 2)
        x_start = max(0, (w_scaled - w) // 2)
        x_end = min(w_scaled, (w_scaled + w) // 2)

        scaled_frame = scaled_frame[y_start:y_end, x_start:x_end]

    # ------- ROTACION -------
    # Asegurarse de que el centro de rotación esté dentro de la imagen
    if rotation_center is None:
        rotation_center = (w // 2, h // 2)  # Por defecto, el centro de la imagen

    M_rotation = cv2.getRotationMatrix2D(rotation_center, rotation_angle, 1)
    rotated_frame = cv2.warpAffine(scaled_frame, M_rotation, (w, h))

    # Mostrar frame transformado
    cv2.imshow('Transformaciones', rotated_frame)

    # Salir si se pulsa la tecla Esc
    if cv2.waitKey(30) & 0xFF == 27: # Esperar 30 ms entre cada frame
        break

cap.release()
cv2.destroyAllWindows()

Nuevo centro de rotación: (244, 109)


-------------------------------------------------------------------------------------------------
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en una imagen destino y realizar la transformación afín. ✔️
- Calcular la imagen especular a partir de una imagen.  ✔️

In [14]:
import cv2
import numpy as np

# Cargar la imagen
image = cv2.imread('02/images/cats.png')
if image is None:
    print("Error: No se pudo cargar la imagen.")
    exit()

# Obtener tamaño de la imagen
h, w = image.shape[:2]

# Variables de los puntos seleccionados
pts1 = []
pts2 = []
selecting_pts1 = True  # Indica si estamos seleccionando puntos de pts1 o pts2
max_points = 3  # Número máximo de puntos por conjunto

# Función de manejo del ratón para la selección de puntos
def select_points(event, x, y, flags, param):
    global pts1, pts2, selecting_pts1
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if selecting_pts1 and len(pts1) < max_points:
            # Agrega el punto a pts1
            pts1.append([x, y])
            print(f"Seleccionado punto {len(pts1)} en pts1: ({x}, {y})")
        elif not selecting_pts1 and len(pts2) < max_points:
            # Agrega el punto a pts2
            pts2.append([x, y])
            print(f"Seleccionado punto {len(pts2)} en pts2: ({x}, {y})")

        # Una vez seleccionados los tres puntos para pts1, cambiamos a pts2
        if len(pts1) == max_points and selecting_pts1 == True:
            selecting_pts1 = False
            print("Seleccionados los puntos de pts1. Ahora selecciona los puntos de pts2.")

# Mostrar la imagen original y configurar el callback del ratón
cv2.imshow('Imagen Original', image)
cv2.setMouseCallback("Imagen Original", select_points)

# Ciclo principal para seleccionar puntos
while True:
    # Mostrar los puntos seleccionados en la imagen
    display_image = image.copy()
    
    # Dibujar los puntos seleccionados de pts1 en azul
    for point in pts1:
        cv2.circle(display_image, tuple(point), 5, (255, 0, 0), -1)  # Azul
    
    # Dibujar los puntos seleccionados de pts2 en rojo
    for point in pts2:
        cv2.circle(display_image, tuple(point), 5, (0, 0, 255), -1)  # Rojo

    # Mostrar la imagen con los puntos seleccionados
    cv2.imshow("Imagen Original", display_image)
    
    # Salir si se han seleccionado todos los puntos
    if len(pts1) == max_points and len(pts2) == max_points:
        break

    cv2.waitKey(1)

# ------- TRANSFORMACIÓN AFIN -------
# Convertir los puntos seleccionados a numpy arrays
pts1 = np.float32(pts1)
pts2 = np.float32(pts2)

# Calcular la transformación afín
Ta = cv2.getAffineTransform(pts1, pts2)

# Aplicar la transformación afín
imageA = cv2.warpAffine(image, Ta, (w, h))

# ------- IMAGEN ESPECULAR -------
# Generar la imagen reflejada horizontalmente (efecto espejo)
mirror_image = cv2.flip(image, 1)  # Flip horizontal (1)

# Mostrar todas las imágenes: original, especular y transformada afín
cv2.imshow('Imagen Especular (Espejo)', mirror_image)
cv2.imshow('Transformación Afin', imageA)

# Esperar a que el usuario presione una tecla para salir
cv2.waitKey(0)
cv2.destroyAllWindows()


Seleccionado punto 1 en pts1: (136, 97)
Seleccionado punto 2 en pts1: (98, 345)
Seleccionado punto 3 en pts1: (763, 61)
Seleccionados los puntos de pts1. Ahora selecciona los puntos de pts2.
Seleccionado punto 1 en pts2: (314, 68)
Seleccionado punto 2 en pts2: (305, 289)
Seleccionado punto 3 en pts2: (648, 140)
